In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import glob
import matplotlib
import matplotlib.pyplot as plt
import dask
from datetime import datetime

In [2]:
# Updated for PBS scheduler
# this could go into utils.
# By default gets 1 core w/ 25 GB memory
def get_ClusterClient(ncores=1, nmem='25GB'):
    import dask
    from dask_jobqueue import PBSCluster
    from dask.distributed import Client
    ncores=ncores
    nmem = nmem

    cluster = PBSCluster(
        cores=ncores, # The number of cores you want
        memory=nmem, # Amount of memory
        processes=ncores, # How many processes
        queue='casper', # The type of queue to utilize (/glade/u/apps/dav/opt/usr/bin/execcasper)
        resource_spec='select=1:ncpus='+str(ncores)+':mem='+nmem, # Specify resources
        project='P93300641', # Input your project ID here
        walltime='1:00:00', # Amount of wall time
        interface='ib0', # Interface to use
    )

    client = Client(cluster)
    return cluster, client

In [3]:
cluster, client = get_ClusterClient()
cluster.scale(20)

In [10]:
client.cluster

### prepare the co2 file

In [3]:
f = '/glade/scratch/djk2120/trendy2022/co2/global_co2_ann_1700_2021.txt'
co2=pd.read_csv(f,header=None,sep="  ",)
co2_from_trendy = co2[1].values

/glade/work/djk2120/miniconda3/envs/trendy-py/lib/python3.7/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [37]:
f = '/glade/p/cgd/tss/people/dll/TRENDY2020_Forcing/fco2_datm_global_simyr_1750-2014_CMIP6_c180929.nc'
ds1 = xr.open_dataset(f,decode_times=False).isel(time=np.arange(320)<319)  #removing 1700
ds1.time.attrs['units']='days since 1701-01-01'

In [40]:
#need to append two more datapoints to the dataset
dt = 365*(1+np.arange(2))
ds2 = ds1.isel(time=[0,1]).copy(deep=True)
ds2['time']=ds1['time'][-1].values+dt

In [42]:
ds3 = xr.concat([ds1,ds2],dim='time')
ds3['CO2'][:]=co2_from_trendy[1:,np.newaxis,np.newaxis]

In [43]:
date = datetime.now().strftime("%m/%d/%Y %H:%M")
hist_str = date+': convert by trendy2022_prep.ipynb'
ds3.attrs={'history':hist_str,'creation_date':date,
              'source':'converted from TRENDYs global_co2_ann_1700_2021.txt',
              'creator':'Daniel Kennedy, djk2120@ucar.edu'}

In [44]:
#take originals, the concat adds a time dimension where it shouldnt
for v in ds1.data_vars:
    if 'time' not in ds1[v].coords:
        ds3.drop(v)
        ds3[v]=ds1[v]

In [45]:
ds3.to_netcdf('/glade/scratch/djk2120/trendy2022/co2/fco2_datm_global_simyr_1701-2021_TRENDY_c220831.nc')

In [46]:
ds3.CO2

<xarray.DataArray 'CO2' (time: 321, lat: 1, lon: 1)>
array([[[276.62]],

       [[276.65]],

       [[276.67]],

       [[276.7 ]],

       [[276.72]],

       [[276.75]],

       [[276.78]],

       [[276.8 ]],

       [[276.83]],

       [[276.86]],

...

       [[392.13]],

       [[394.88]],

       [[396.94]],

       [[399.24]],

       [[402.52]],

       [[404.71]],

       [[406.94]],

       [[409.63]],

       [[412.1 ]],

       [[414.27]]], dtype=float32)
Coordinates:
  * time     (time) float64 181.0 546.0 911.0 ... 1.163e+05 1.166e+05 1.17e+05
  * lat      (lat) float64 0.0
  * lon      (lon) float64 180.0
Attributes:
    coordinate:  latc lonc time
    units:       ppmv
    lname:       CO2 concentration

### edit the ndep files

In [28]:
f = glob.glob('/glade/scratch/djk2120/trendy2022/ndep/*.nc')
ds2 = xr.open_mfdataset(f,combine='nested',decode_times=False,parallel=True,chunks={'time':12})

In [7]:
f = '/glade/p/cgd/tss/people/dll/TRENDY2019_Forcing/Ndep/TRENDY2019_Ndep_Monthly_c190821.nc'
ds = xr.open_dataset(f)

In [30]:
ndep = xr.Dataset()
clm_names = ['dry_deposition_NHx_as_N','dry_deposition_NOy_as_N','wet_deposition_NHx_as_N','wet_deposition_NOy_as_N']
trn_names = ['drynhx','drynoy','wetnhx','wetnoy']

for clm,trn in zip(clm_names,trn_names):
    ndep[clm]=ds2[trn].copy(deep=True)
    

In [31]:
noy = 'NDEP_NOy_month'
ndep[noy] = 1000*(ds2.drynoy+ds2.wetnoy).copy(deep=True)
ndep[noy].attrs = ds[noy].attrs

nhx = 'NDEP_NHx_month'
ndep[nhx] = 1000*(ds2.drynhx+ds2.wetnhx).copy(deep=True)
ndep[nhx].attrs = ds[nhx].attrs

ntot = 'NDEP_month'
ndep[ntot] = ndep[noy]+ndep[nhx]
ndep[ntot].attrs = ds[ntot].attrs
ndep['time']=ds['time'].copy(deep=True)

In [33]:
thedir = '/glade/scratch/djk2120/trendy2022/ndep/'
nfile  = thedir+'TRENDY2022_Ndep_Monthly_c220802.nc'
ndep['dry_deposition_NHx_as_N'].to_netcdf(nfile)

In [34]:
appends = list(ndep.data_vars)
appends.remove('dry_deposition_NHx_as_N')
for thisvar in appends:
    print(thisvar)
    ndep[thisvar].to_netcdf(nfile,'a')

dry_deposition_NOy_as_N
wet_deposition_NHx_as_N
wet_deposition_NOy_as_N
NDEP_NOy_month
NDEP_NHx_month
NDEP_month


In [144]:
da = ds.datesec.copy(deep=True)
da[:]=0
ndep['datesec'] = da.copy(deep=True)

In [145]:
ndep['date']=ds['date'].copy(deep=True)

In [149]:
gw = np.cos(ndep.lat/360*2*np.pi)
gw = 2*gw/gw.sum()
ndep['gw'] = xr.DataArray(gw,dims='lat')

In [162]:
appends = ['date','datesec','gw']
for thisvar in appends:
    print(thisvar)
    ndep[thisvar].to_netcdf(nfile,'a')

date
datesec
gw
